# Brainstorming:

Problem: 
- User has large pdf file that has factory test data. 
- PDF file is too large: 22k pages
- Data is in a table format
- Data inside table is messy, need parsing/cleaning

General solution:
- Use python to parse pdf file
- Get data from pdf file, parse data in desire format, export into excel file for visualization later

Specific solution, pipeline:
- Use camelot to parse tables from pdf files
- Detect desire table, concat them.
- Cleaning and organizing data into a nice/correct df:
  - Check and drop extra column (Done)
  - Detect and concat tables
  - Seperate first cell into extra columns
- Export df into excel file

Design note:
- Hardcoded: check all tables, if tables 7 columns: drop first one: all tables should have 6



# Working code on small file:


In [9]:
import camelot
import pandas as pd
import numpy as np
from IPython.display import display
import time
import re
from PyPDF2 import PdfReader

def read_pdf(file_path):
    # Read the PDF file using Camelot
    print(f"Start reading PDF file {file_path}...")
    tables = camelot.read_pdf(file_path, pages='all' )
    # clean_tables(tables)
    return tables

def clean_table(table):
    print(f"\nStart cleaning table...")
    # Check if the table has more than 1 column
    if table.shape[1] > 1:
        print(f'Table has {table.shape[1]} columns. Cleaning up...')
        # # Drop the first column
        table = table.drop(columns=0)
        # Reset the column names
        table.columns = range(table.shape[1])
        print(f'Table now has {table.shape[1]} columns.')
    return table

# Find target_table location in the PDF file: 
# return a list of page numbers
def find_table_location(pdf_file, target_table):
    print(f"Searching for table: {target_table}")
    
    reader = PdfReader(pdf_file)

    pages_with_table = []

    for page_num in range(len(reader.pages)):

        print(f"Searching page {page_num} out of {len(reader.pages)}")
        
        page = reader.pages[page_num]
        text = page.extract_text()
        
        if target_table in text:
            print(f"Found table on page {page_num}")
            pages_with_table.append(page_num)

            # Check for multi-page
            if page_num < len(reader.pages)-2:

                # Get page_num+2 
                next_page = reader.pages[page_num+2]  
                next_text = next_page.extract_text()

                if re.search(r'\d+\.\d+\.\d+', next_text):
                    pages_with_table.append(page_num + 2)

            # Add extra page    
            if pages_with_table:
                pages_with_table.append(pages_with_table[-1] + 2)

    print(f"Table pages found: {pages_with_table}")
                
    return pages_with_table

def convert_to_ranges(numbers):
    """
    Convert a list of numbers into a list of ranges.
    """
    ranges = []
    for i in range(0, len(numbers), 2):
        ranges.append(f"{numbers[i]}-{numbers[i+1]}")
    return ranges

def display_tables(tables):
    # Set the max column width to a high number (e.g., 1000) to display long contents
    pd.set_option('display.max_colwidth', 1000)

    # Display all tables
    print(f"Displaying {len(tables)} tables:")
    for i, table in enumerate(tables):
        table_number = i + 1
        print(f"\nTable {table_number}")
        display(table.df)

def display_processed_tables(tables):
    # Set the max column width to a high number (e.g., 1000) to display long contents
    pd.set_option('display.max_colwidth', 1000)

    # Display all processed tables
    print(f"\nDisplaying {len(tables)} tables:")
    for i, df in enumerate(tables):
        print(f"Table {i + 1}")
        display(df)

def extract_band(tables):
    print(f"Extracting Band info...")
    for table in tables:
        line2 = table.iloc[0,0]
        if 'Band' in line2:
            band_info = line2.split(' ')
            for word in band_info:
                if word.startswith('Band'):
                    band_num = word[4:]  # Extract everything after "Band"
                    table.insert(1, 'Band', band_num)
                    break
            else:
                print(f"No 'Band' keyword in line: {line2}")
    return tables

def extract_info(tables):
    print(f"Extracting Testname, ULCH, BW, MOD, RD info...")
    for i, table in enumerate(tables):
        # Define patterns
        testname_pattern = r"(.*):@"
        ulch_pattern = r"ULCH: (\d+),"
        bw_pattern = r"BW: ([\d\.]+ MHz)"
        mod_pattern = r"UL_MOD_RB: ([^,]+),"
        rd_pattern = r"UL_MOD_RB: [^,]+, (.*)"

        # Extract info
        table['Testname'] = table.iloc[:,0].str.extract(testname_pattern)
        table['ULCH'] = table.iloc[:,0].str.extract(ulch_pattern)
        table['BW'] = table.iloc[:,0].str.extract(bw_pattern)
        table['MOD'] = table.iloc[:,0].str.extract(mod_pattern)
        table['RD'] = table.iloc[:,0].str.extract(rd_pattern)

        # Drop the first column
        table.drop(table.columns[0], axis=1, inplace=True)

        # Reorganize the columns
        print(f"Reorganizing columns...")
        new_column_order = ['Testname', 'Band', 'ULCH', 'BW', 'MOD', 'RD', 'Limit Low', 'Limit High', 'Measured', 'Unit', 'Status']
        table = table.reindex(columns=new_column_order)

        # Replace the table in the list with the cleaned and reorganized table
        tables[i] = table
        
    return tables    

# Find target_table and concatenate with continuous tables 
# return a list of concatenated tables
def find_target_table(tables, desired_name):
    
    current_table = None
    desired_tables = []
    
    #Run through table_list
    for table in tables:
        first_cell = table.df.iloc[0,0].split('\n')[0]

        #If the cell in first row, first column has the desire format "6.x.x.x" AND the desire_name: Start a new table
        if first_cell.startswith("6.") and desired_name in first_cell:
            # Found start of new desired table
            print(f"Found start of new desired table: {first_cell}")
            if current_table is None:
                current_table = table
            current_table = table.df.copy()    
            
        elif first_cell.startswith("6.") and desired_name not in first_cell: 
            #else: return the current table, and reset the current_table to None
            print(f"\nSkipping a none-desired table: {first_cell}")
            if current_table is not None:
                # Save the current table
                desired_tables.append(current_table)
                #Reset, mark end of the desired table
                current_table = None
        
        #if the table doesnt match the desire_format AND there is a current_table: concat this table into the current table:         
        elif current_table is not None:
            # Continuation of previous desired table
            print(f"Found continuation of previous desired table: {table.df.iloc[0,0]}")
            # print(f"Table before clean: ")
            # display(table.df)
            #Clean table before concat:
            table.df = clean_table(table.df)
            current_table = pd.concat([current_table, table.df])

            # print(f"Table after concat:")
            # display(current_table)
        
            
    return desired_tables

# Clean, extract info in raw tables:
# return a list of cleaned tables
def process_tables(tables):  

    processed_tables = []
    
    for i, table in enumerate(tables):
        print(f"\nProcessing table {i+1}...")
        # Replace all-whitespace rows with NaN
        table = table.replace(r'^\s*$', np.nan, regex=True)
        # Drop the rows where all elements are NaN
        table = table.dropna(how='all')
        # print("Table after drop empty row:")
        # display(table)
    
        # Split the first cell of the first row and use it as column headers
        first_row = table.iloc[0, 0]
        
        headers = first_row.split('\n')  
        # print(f"First row: {first_row}")
        # print(f"Headers: {headers}")
        # headers.append("MissingHeader")
        table.columns = headers
        
        # Extract the table name from the first row
        table_name = table.iloc[0, 0].split('\n')[0]

        # Remove the first row
        table = table.iloc[1:]
        
        # Extract Band info
        print(f"Extracting Band info...")
        line2 = table.iloc[0,0]
        # print(f"line2: {line2}")
        if 'Band' in line2:
            band_info = line2.split(' ')
            for word in band_info:
                if word.startswith('Band'):
                    band_num = word[4:]  # Extract everything after "Band"
                    table.insert(1, 'Band', band_num)
                    break
            else:
                print(f"No 'Band' keyword in line: {line2}")
        # Drop the first row
        table = table.drop(table.index[0])
        # print("Table after header:")
        # display(table)
          
        # Check if the expected columns are in the table
        if 'Unit' not in table.columns:
            print(f"Table {i+1} doesn't have the expected structure. Skipping...")
            continue
  
        # Extract Testname, ULCH, BW, MOD, RD info
        print(f"Extracting Testname, ULCH, BW, MOD, RD info...")
        # Define patterns
        testname_pattern = r"(.*):@"
        ulch_pattern = r"ULCH: (\d+),"
        bw_pattern = r"BW: ([\d\.]+ MHz)"
        mod_pattern = r"UL_MOD_RB: ([^,]+),"
        rd_pattern = r"UL_MOD_RB: [^,]+, (.*)"

        # Extract info
        table['Testname'] = table.iloc[:,0].str.extract(testname_pattern)
        table['ULCH'] = table.iloc[:,0].str.extract(ulch_pattern)
        table['BW'] = table.iloc[:,0].str.extract(bw_pattern)
        table['MOD'] = table.iloc[:,0].str.extract(mod_pattern)
        table['RD'] = table.iloc[:,0].str.extract(rd_pattern)

        # Create a temporary DataFrame for the split results
        split_df = table['Unit'].str.split(expand=True)

        # Assign the split results to 'Measured' and 'Unit' only where there are values
        table.loc[split_df[0].notna(), 'Measured'] = split_df.loc[split_df[0].notna(), 0]
        table.loc[split_df[1].notna(), 'Unit'] = split_df.loc[split_df[1].notna(), 1]

        # Drop the first column
        table.drop(table.columns[0], axis=1, inplace=True)

        # Create a new column filled with the table name
        table.insert(0, 'Table Name', table_name)
        table = table.iloc[1:]
        
        # Reorganize the columns
        print(f"Reorganizing columns...")
        new_column_order = ['Table Name', 'Testname', 'Band', 'ULCH', 'BW', 'MOD', 'RD', 'Limit Low', 'Limit High', 'Measured', 'Unit', 'Status']
        table = table.reindex(columns=new_column_order)

        # Reset the index
        table.reset_index(drop=True, inplace=True)
        
        #Append processed table to processed_tables:
        processed_tables.append(table)
        
        
    return processed_tables


short_file = "2_pages_test.pdf"

# "LTE_3GPP_v15_r8_FDD_FORD_All_TEMPS_TCU2_5_ROW_012023_5GSIM_AT_2023-06-28_16-27-57_188.pdf"
#  "25C_DATA_Extract(1temperatureOnly).pdf"
# "300_pages_extract.pdf"
long_file = "LTE_3GPP_v15_r8_FDD_FORD_All_TEMPS_TCU2_5_ROW_012023_5GSIM_AT_2023-06-28_16-27-57_188.pdf"
target_table = "6.2.2 Maximum Output Power" 




# Run on large file:

In [ ]:
# Find target table pages
pages = find_table_location(long_file, target_table)

# Format page numbers for camelot
page_ranges = convert_to_ranges(pages)
print(f"Pages to read: {page_ranges}")



In [146]:
print(f"Pages to read: {page_ranges}")


Pages to read: ['3-5', '291-293', '815-817', '1212-1214', '1736-1738', '2072-2074', '2358-2360', '2694-2696', '2980-2982', '3375-3377', '3660-3662', '3945-3947', '4437-4439', '4726-4728', '5250-5252', '5647-5649', '6171-6173', '6507-6509', '6793-6795', '7129-7131', '7415-7417', '7810-7812', '8095-8097', '8380-8382', '8872-8874', '9161-9163', '9685-9687', '10082-10084', '10606-10608', '10942-10944', '11228-11230', '11564-11566', '11850-11852', '12245-12247', '12530-12532', '12815-12817', '13307-13309', '13596-13598', '14120-14122', '14517-14519', '15041-15043', '15377-15379', '15663-15665', '15999-16001', '16285-16287', '16680-16682', '16965-16967', '17250-17252', '17742-17744', '18030-18032', '18554-18556', '18951-18953', '19475-19477', '19811-19813', '20097-20099', '20433-20435', '20719-20721', '21114-21116', '21399-21401', '21684-21686']


In [ ]:
clean_tables = []
   
for page_range in page_ranges:
    print(f"\nProcessing page range: {page_range}")
 
    # Read all tables from pdf
    long_tables = camelot.read_pdf(long_file, pages=page_range)

    desire_tables = find_target_table(long_tables, "6.2.2 Maximum Output Power")
    # display_processed_tables(desire_tables)

    processed_tables = process_tables(desire_tables)
    #append to clean_tables:
    clean_tables.extend(processed_tables)


display_processed_tables(clean_tables)

In [ ]:
# Concatenate all the dataframes in the list into a single dataframe
all_tables = pd.concat(clean_tables, ignore_index=True)
all_tables.head(10) # Display the first few rows of the resulting dataframe

# Testing


In [ ]:
with open("22kPDF.txt", "r") as file:
    page_ranges = file.read().splitlines()
page_ranges

In [11]:
# Find target_table and concatenate with continuous tables 
# return a list of concatenated tables
def find_target_table(tables, desired_name):
    
    current_table = None
    desired_tables = []
    
    #Run through table_list
    for table in tables:
        first_cell = table.df.iloc[0,0].split('\n')[0]

        #If the cell in first row, first column has the desire format "6.x.x.x" AND the desire_name: Start a new table
        if first_cell.startswith("6.") and desired_name in first_cell:
            # Found start of new desired table
            
            if current_table is None:
                current_table = table
            current_table = table.df.copy()
            # print(f"Found start of new desired table:")   
            # display(current_table)
        elif first_cell.startswith("6.") and desired_name not in first_cell: 
            #else: return the current table, and reset the current_table to None
            print(f"\nSkipping a none-desired table: {first_cell}")
            if current_table is not None:
                # Save the current table
                desired_tables.append(current_table)
                #Reset, mark end of the desired table
                current_table = None
        
        #if the table doesnt match the desire_format AND there is a current_table: concat this table into the current table:         
        elif current_table is not None:
            # Continuation of previous desired table
            # print(f"Found continuation of previous desired table: {table.df.iloc[0,0]}")
            # print(f"Found continuous table, before cleaning:")   
            # display(table.df)
            #Clean table before concat:
            table.df = clean_table(table.df)
            # print(f"Continuous table after cleaning:")   
            # display(table.df)
            current_table = pd.concat([current_table, table.df])

            # print(f"Table after concat:")
            # display(current_table)
        
            
    return desired_tables

# Clean, extract info in raw tables:
# return a list of cleaned tables
def process_tables(tables):  

    processed_tables = []
    
    for i, table in enumerate(tables):
        print(f"\nProcessing table {i+1}...")
        print(f"Table before processing:")
        display(table)

    
        # Split the first cell of the first row and use it as column headers
        first_row = table.iloc[0, 0]
        headers = first_row.split('\n')  
        # print(f"First row: {first_row}")
        # print(f"Headers: {headers}")
        # headers.append("MissingHeader")
        table.columns = headers
        
        # Extract the table name from the first row
        table_name = table.iloc[0, 0].split('\n')[0]

        
        # Extract Band info from column 2
        print(f"Extracting Band info...")
        line2 = table.iloc[1,0]
        print(f"line2: {line2}")
        if 'Band' in line2:
            band_info = line2.split(' ')
            for word in band_info:
                if word.startswith('Band'):
                    band_num = word[4:]  # Extract everything after "Band"
                    table.insert(1, 'Band', band_num)
                    break
            else:
                print(f"No 'Band' keyword in line: {line2}")
         

        # print("Table after header:")
        # display(table)
          
        # # Check if the expected columns are in the table
        # if 'Unit' not in table.columns:
        #     print(f"Table {i+1} doesn't have the expected structure. Skipping...")
        #     continue
  
        # Extract Testname, ULCH, BW, MOD, RD info
        print(f"Extracting Testname, ULCH, BW, MOD, RD info...")
        # Define patterns
        testname_pattern = r"(.*):@"
        ulch_pattern = r"ULCH: (\d+),"
        bw_pattern = r"BW: ([\d\.]+ MHz)"
        mod_pattern = r"UL_MOD_RB: ([^,]+),"
        rd_pattern = r"UL_MOD_RB: [^,]+, (.*)"

        # Extract info
        table['Testname'] = table.iloc[:,0].str.extract(testname_pattern)
        table['ULCH'] = table.iloc[:,0].str.extract(ulch_pattern)
        table['BW'] = table.iloc[:,0].str.extract(bw_pattern)
        table['MOD'] = table.iloc[:,0].str.extract(mod_pattern)
        table['RD'] = table.iloc[:,0].str.extract(rd_pattern)
        
        # print(f"Table before split Unit column:")
        # display(table)
   
        # Split 'Measured' and 'Unit' columns 
        # Create a temporary DataFrame for the split results
        split_df = table['Unit'].str.split(expand=True)

        # Assign the split results to 'Measured' and 'Unit' only where there are values
        table.loc[split_df[0].notna(), 'Measured'] = split_df.loc[split_df[0].notna(), 0]
        table.loc[split_df[1].notna(), 'Unit'] = split_df.loc[split_df[1].notna(), 1]

        # Drop the first column
        table.drop(table.columns[0], axis=1, inplace=True)

        # Create a new column filled with the table name
        table.insert(0, 'Table Name', table_name)
        table = table.iloc[1:]
        
        # Reorganize the columns
        print(f"Reorganizing columns...")
        new_column_order = ['Table Name', 'Testname', 'Band', 'ULCH', 'BW', 'MOD', 'RD', 'Limit Low', 'Limit High', 'Measured', 'Unit', 'Status']
        table = table.reindex(columns=new_column_order)

        # Drop rows with NaN values in Testname and Band columns
        table = table.dropna(subset=['Testname'], how='all')


        # Reset the index
        table.reset_index(drop=True, inplace=True)
        
        #Append processed table to processed_tables:
        processed_tables.append(table)
        
        
    return processed_tables


clean_tables = []
total_ranges = len(page_ranges)
for i, page_range in enumerate(page_ranges):
    
    print(f"Processing page range {i+1}/{total_ranges}...")
    # Calculate the estimate time remaining based on the progress so far
    progress = i / total_ranges
    time_remaining = (total_ranges - i) * 9 / 60
    print(f"Estimated time remaining: {time_remaining:.2f} minutes")
    
    start_time = time.time()
    # Read all tables from pdf
    long_tables = camelot.read_pdf(long_file, pages=page_range)
    end_time = time.time()
    print(f"Time taken to load table: {end_time - start_time:.2f} seconds")
    
    start_time = time.time()
    desire_tables = find_target_table(long_tables, "6.2.2 Maximum Output Power")
    # display_processed_tables(desire_tables)
    end_time = time.time()
    print(f"Time taken to find target table: {end_time - start_time:.2f} seconds")
   
    
    start_time = time.time()
    processed_table = process_tables(desire_tables)
    end_time = time.time()
    print(f"Time taken to process tables: {end_time - start_time:.2f} seconds")
    display_processed_tables(processed_table)
    
    
    #append to clean_tables:
    clean_tables.extend(processed_table)
    



Processing page range 1/60...
Estimated time remaining: 9.00 minutes


GhostscriptError: -100

In [144]:
display(all_tables.head(10)) # Display the first few rows of the resulting dataframe

# Concatenate all the dataframes in the list into a single dataframe
all_tables = pd.concat(clean_tables, ignore_index=True)

# Export the dataframe to an Excel file
all_tables.to_excel("all_tables.xlsx", index=False)

,Table Name,Testname,Band,ULCH,BW,MOD,RD,Limit Low,Limit High,Measured,Unit,Status
0,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18607,1.4 MHz,QPSK,1 (RB_Pos:LOW),18.80,25.70,22.71,dBm,Passed
1,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18900,1.4 MHz,QPSK,1 (RB_Pos:LOW),20.30,25.70,22.80,dBm,Passed
2,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18900,1.4 MHz,QPSK,5 (RB_Pos:LOW),20.30,25.70,22.82,dBm,Passed
3,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,19193,1.4 MHz,QPSK,1 (RB_Pos:LOW),18.80,25.70,22.61,dBm,Passed
4,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18615,3.0 MHz,NaN,NaN,,,Not Required for\n3GPP default,,Passed
5,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18900,3.0 MHz,NaN,NaN,,,Not Required for\n3GPP default,,Passed
6,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,19185,3.0 MHz,NaN,NaN,,,Not Required for\n3GPP default,,Passed
7,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18625,5.0 MHz,QPSK,1 (RB_Pos:LOW),18.80,25.70,22.80,dBm,Passed
8,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18625,5.0 MHz,QPSK,1 (RB_Pos:HIGH),20.30,25.70,22.86,dBm,Passed
9,6.2.2 Maximum Output Power,UE Maximum Output Power,NaN,18900,5.0 MHz,QPSK,1 (RB_Pos:LOW),20.30,25.70,22.84,dBm,Passed
